In [2]:
import pickle

import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta
from collections import defaultdict

import networkx as nx
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import pyvis

import networkx as nx
from pyvis.network import Network

import seaborn as sns
import matplotlib.pyplot as plt

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

## Loading data

In [3]:
single_experience_nodes_by_month_and_no_job_periods_df = pd.read_parquet("../data/processed/single_experience_nodes_by_month_and_no_job_periods.parquet")
print(f"Shape: {single_experience_nodes_by_month_and_no_job_periods_df.shape}")
single_experience_nodes_by_month_and_no_job_periods_df

Shape: (817075, 6)


,date,person_id,company_id,start_date,end_date,model_classification
0,2010-01-01,0.0,2.0,2010-01-01,2024-05-01,030000
1,2010-01-01,0.0,3.0,2010-01-01,2017-01-01,192041
2,2010-02-01,0.0,2.0,2010-01-01,2024-05-01,030000
3,2010-02-01,0.0,3.0,2010-01-01,2017-01-01,192041
4,2010-03-01,0.0,2.0,2010-01-01,2024-05-01,030000
...,...,...,...,...,...,...
19,2024-01-01,9245.0,652.0,2024-01-01,2024-05-01,132051
20,2024-02-01,9245.0,652.0,2024-01-01,2024-05-01,132051
21,2024-03-01,9245.0,652.0,2024-01-01,2024-05-01,132051
22,2024-04-01,9245.0,652.0,2024-01-01,2024-05-01,132051


# Graph option 1: time dimensional graph, untraceable career paths

- Each node represents a given job occupation at a given time;
- Each vertex represents a transition from job occupation A at time instant t1 to job occupation B at time instant t2

- Multiple jobs per node are NOT allowed

In [3]:
def create_directed_temporal_graph(input_df, job_titles, time_dim):

    G = nx.DiGraph()

    for person_id in sorted(input_df['person_id'].unique()): # all jobs occupations of a given person
        person_df = input_df[input_df['person_id'] == person_id]

        for date0 in sorted(person_df[time_dim].unique()): # all job occupations of a given person in a given month
            occupations_date0_df = person_df[person_df[time_dim] == date0]
            occupations_date0 = set(occupations_date0_df[job_titles])

            date1 = date0 + pd.DateOffset(months=1)
            occupations_date1_df = person_df[person_df[time_dim] == date1]
            occupations_date1 = set(occupations_date1_df[job_titles])

            date2 = date0 + pd.DateOffset(months=2)
            occupations_date2_df = person_df[person_df[time_dim] == date2]
            occupations_date2 = set(occupations_date2_df[job_titles])
            
            included_occupations_date0 = []
            for _, current_occupation in occupations_date0_df.iterrows(): # one job occupation of a given person in a given month
                if current_occupation[job_titles] not in included_occupations_date0:
                    date0_str = pd.to_datetime(str(date0)).strftime('%Y-%m-%d')
                    node_current = current_occupation[job_titles] + ' - ' + date0_str
                    
                    included_occupations_date0.append(current_occupation[job_titles])

                    if not G.has_node(node_current):
                        G.add_node(node_current, date=date0_str, occupation=current_occupation[job_titles], weight=1)
                    else:
                        G.nodes[node_current]['weight'] += 1
                    
                    if len(occupations_date1) > 0:
                        included_occupations_date1 = []
                        for _, next_occupation in occupations_date1_df.iterrows(): # one job occupation of a given person in the next month
                            if next_occupation[job_titles] not in included_occupations_date1:
                                next_date_str = pd.to_datetime(str(date1)).strftime('%Y-%m-%d')
                                node_next = next_occupation[job_titles] + ' - ' + next_date_str

                                included_occupations_date1.append(next_occupation[job_titles])
                            
                                if not G.has_node(node_next):
                                    G.add_node(node_next, date=next_date_str, occupation=next_occupation[job_titles], weight=0)

                                if not G.has_edge(node_current, node_next):
                                    
                                    # Condition 1: person has 1 job in current month -> connect to all jobs in the next month
                                    if (len(occupations_date0) == 1):
                                        G.add_edge(node_current, node_next, weight=1)

                                    # Condition 2: person has >1 jobs in current month and only 1 job in the next month
                                    # next_occupations = set(occupations_date1_df[job_titles])
                                    intersec_occupations = occupations_date0 & occupations_date1
                                    all_occupations = occupations_date0 | occupations_date1
                                    different_occupations = all_occupations - intersec_occupations
                                    if (occupations_date0_df.shape[0] > 1) \
                                    and (occupations_date1_df.shape[0] == 1):
                                        # Condition 2.1: one of the jobs in the current month is the same as the one in the next month -> connect only the identical jobs
                                        if current_occupation[job_titles] in intersec_occupations \
                                        and next_occupation[job_titles] in intersec_occupations:
                                            G.add_edge(node_current, node_next, weight=1)
                                        # Condition 2.2: no job of the current month is the same as the one in the next month -> connect all current jobs to the next job
                                        elif len(intersec_occupations) == 0:
                                            G.add_edge(node_current, node_next, weight=1)

                                    # Condition 3: person has >1 jobs in current month (X and Y) and >1 jobs in the next month (X and Z) AND at least one job is the same between the two months (X) -> connect job X to X and Y to Z
                                    if (occupations_date0_df.shape[0] > 1) \
                                    and (occupations_date1_df.shape[0] > 1) \
                                    and (len(intersec_occupations) >= 1):
                                        
                                        if current_occupation[job_titles] in intersec_occupations \
                                        and next_occupation[job_titles] in intersec_occupations \
                                        and current_occupation[job_titles] == next_occupation[job_titles]:
                                            G.add_edge(node_current, node_next, weight=1)

                                        elif current_occupation[job_titles] in different_occupations \
                                        and next_occupation[job_titles] in different_occupations:
                                            G.add_edge(node_current, node_next, weight=1)

                                        if current_occupation[job_titles] in intersec_occupations \
                                        and next_occupation[job_titles] in different_occupations \
                                        and current_occupation[job_titles] not in occupations_date2:
                                            G.add_edge(node_current, node_next, label=str(person_id), weight=1)

                                    # Condition 4: person has >1 jobs in current month (X and Y) and >1 jobs in the next month (Z and K) AND no job is the same between the two months -> connect all
                                    if (occupations_date0_df.shape[0] > 1) \
                                    and (occupations_date1_df.shape[0] > 1)\
                                    and (len(intersec_occupations) == 0):
                                        G.add_edge(node_current, node_next, weight=1)

                                else:
                                    G[node_current][node_next]['weight'] += 1
                        

    return G

# df_in_test = df_in[df_in['person_id'].isin([3, 137])] # 137, 3 - 3, 5, 11, 13, 14, 137
gf_temporal_single_experience_nodes_untraceable = create_directed_temporal_graph(single_experience_nodes_by_month_and_no_job_periods_df, 'model_classification', 'date')

In [25]:
with open("gf_temporal_single_experience_nodes_untraceable.pkl", "wb") as f:
    pickle.dump(gf_temporal_single_experience_nodes_untraceable, f)

In [6]:
with open(r"gf_temporal_single_experience_nodes_untraceable.pkl", "rb") as input_file:
    gf_temporal_single_experience_nodes_untraceable = pickle.load(input_file)
gf_temporal_single_experience_nodes_untraceable

In [7]:
pyvis_net = Network(height="720px", width="100%", bgcolor='#FFFFFF', font_color='black', directed=True, notebook=True, select_menu=True, filter_menu=True)
pyvis_net.toggle_physics(False)

# Set up positions
occupations = sorted(set(nx.get_node_attributes(gf_temporal_single_experience_nodes_untraceable, 'occupation').values()))
dates = sorted(set(nx.get_node_attributes(gf_temporal_single_experience_nodes_untraceable, 'date').values()))
occupation_positions = {occ: i * 400 for i, occ in enumerate(occupations)}
date_positions = {date: i * 400 for i, date in enumerate(dates)}

# Add nodes with fixed positions and scaled sizes
for node, node_attrs in gf_temporal_single_experience_nodes_untraceable.nodes(data=True):
    x = date_positions[node_attrs['date']]
    y = occupation_positions[node_attrs['occupation']]
    weight = node_attrs.get('weight', 1)
    label = f"{node_attrs['occupation']}\n{node_attrs['date']}"
    title = label + f"\nNumber of people: {node_attrs.get('weight', 1)}"
    pyvis_net.add_node(node,  x=x, y=y, 
                       label=label, title=title,
                       color=f'rgba(0,0,0,1)',
                       size=np.log10(weight)*8,
                       occupation=node_attrs['occupation'], 
                       date=node_attrs['date'], 
                       number_of_people=weight)


# Add edges with scaled widths
for source, target, edge_attrs in gf_temporal_single_experience_nodes_untraceable.edges(data=True):
    weight = edge_attrs.get('weight', 1)
    title = f"Number of people: {weight}"
    pyvis_net.add_edge(source, target, 
                       title=title,
                       width=np.log10(weight), 
                       arrowStrikethrough=False,
                       color=f'rgba(150,150,150,1)',
                       number_of_people=weight
                       )


pyvis_net.save_graph('time_dimensional_single_experience_nodes_untraceable.html')

In [5]:
# Read the saved HTML and add the filter dropdown
with open('time_dimensional_single_experience_nodes_untraceable.html', 'r') as file:
    html_content = file.read()

#######################################
##  CUSTOM YEAR MARKERS

custom_css = """ 
<style>
.year-label {
    position: absolute;
    color: black;
    transform: translateX(-50%);
    z-index: 10;
    top: 10px;
}
.occupation-label {
    position: absolute;
    color: black;
    transform: translateY(-50%);
    z-index: 10;
    right: 10px;
}
</style>
"""

custom_js = """
<script type="text/javascript">
var markers = {{{};
var occupationMarkers = {};

function parseDateUTC(dateString) {
    var parts = dateString.split('-');
    return new Date(Date.UTC(parts[0], parts[1] - 1, parts[2]));
}

function addYearLabels() {
    var container = document.getElementById('mynetwork');

    nodes.forEach(function(node) {
        var date = parseDateUTC(node.date);
        var year = date.getUTCFullYear();
        var month = date.getUTCMonth();

        if (!markers[year]) {
            markers[year] = { first: null };
        }
        if (month === 0 && (!markers[year].first || date < parseDateUTC(markers[year].first.date))) {
            markers[year].first = node; // Node with January date
        }
    });

    Object.keys(markers).forEach(function(year) {
        if (markers[year].first) {
            var firstLabel = document.createElement('div');
            firstLabel.className = 'year-label';
            firstLabel.id = 'year-label-first-' + year;
            firstLabel.innerHTML = year;
            container.appendChild(firstLabel);
        }
    });
}

function addOccupationLabels() {
    var container = document.getElementById('mynetwork');

    nodes.forEach(function(node) {
        var occupation = node.occupation;

        if (!occupationMarkers[occupation]) {
            occupationMarkers[occupation] = { nodes: [] };
        }
        occupationMarkers[occupation].nodes.push(node);
    });

    Object.keys(occupationMarkers).forEach(function(occupation) {
        if (occupationMarkers[occupation].nodes.length > 0) {
            var label = document.createElement('div');
            label.className = 'occupation-label';
            label.id = 'occupation-label-' + occupation;
            label.innerHTML = occupation;
            container.appendChild(label);
        }
    });
}

function updateYearLabels() {
    var container = document.getElementById('mynetwork');
    var rect = container.getBoundingClientRect();

    Object.keys(markers).forEach(function(year) {
        if (markers[year].first) {
            var firstLabel = document.getElementById('year-label-first-' + year);
            var firstNodePosition = network.getPositions([markers[year].first.id])[markers[year].first.id];
            var firstCanvasPos = network.canvasToDOM({ x: firstNodePosition.x, y: 0 });

            firstLabel.style.left = (firstCanvasPos.x + 15) + 'px';
            firstLabel.style.top = '10px'; // Set the y position for the first label
        }
    });
}



document.addEventListener("DOMContentLoaded", function() {
    addYearLabels();
    addOccupationLabels();

    network.on('afterDrawing', function() {
        updateYearLabels();
        updateOccupationLabels();
    });

    updateYearLabels(); // Initial positioning of the year labels
    updateOccupationLabels(); // Initial positioning of the occupation labels
});
</script>
"""

html_content = html_content.replace('</head>', f'{custom_css}</head>')
html_content = html_content.replace('</body>', f'{custom_js}</body>')

with open('time_dimensional_single_experience_nodes_untraceable.html', 'w') as file:
    file.write(html_content)

# Graph option 2: time dimensional graph with traceable career path

It's a MultiDiGraph, so it's possible to have multiple edges between the same two nodes, varying only their attributes. I use it to track specific career tracks of people by adding the person_id (anonymized) to the edge.

- Each node represents a given job occupation at a given time;
- Each edge represents a given person transitioning from job occupation A at time instant t1 to job occupation B at time instant t2

- Multiple jobs per node are NOT ALLOWED

In [6]:
def add_edge_update_label(G, node_current, node_next, person_id):

    if G.has_edge(node_current, node_next):
        existing_label = G[node_current][node_next][0]['label']
        existing_person_ids = set(existing_label.split(', '))
        if str(person_id) not in existing_person_ids:
            new_label = existing_label + f", {person_id}"
            G[node_current][node_next][0]['label'] = new_label
            G[node_current][node_next][0]['person_id'] = new_label
        G[node_current][node_next][0]['weight'] += 1
    else:
        G.add_edge(node_current, node_next, label=str(person_id), weight=1, person_id=str(person_id))

def create_directed_temporal_graph(input_df, job_titles, time_dim):

    G = nx.MultiDiGraph()
    
    for person_id in sorted(input_df['person_id'].unique()): # we will iterate for each person
        person_df = input_df[input_df['person_id'] == person_id] # select all job occupations of that person

        for date0 in sorted(person_df[time_dim].unique()): # select all months that the person was employed
            occupations_date0_df = person_df[person_df[time_dim] == date0] # jobs in month X that the person had
            occupations_date0 = set(occupations_date0_df[job_titles])

            date1 = date0 + pd.DateOffset(months=1)
            occupations_date1_df = person_df[person_df[time_dim] == date1] # jobs in month X+1 that the person had
            occupations_date1 = set(occupations_date1_df[job_titles])

            # date4 = date0 + pd.DateOffset(months=4)
            # occupations_date4_df = person_df[person_df[time_dim] == date4] # jobs in month X+4 that the person had
            # occupations_date4 = set(occupations_date4_df[job_titles])
            
            included_occupations_date0 = []
            for _, current_occupation in occupations_date0_df.iterrows(): # one job occupation of that person in a given month
                if current_occupation[job_titles] not in included_occupations_date0:
                    date_str = pd.to_datetime(str(date0)).strftime('%Y-%m-%d')
                    node_current = current_occupation[job_titles] + ' - ' + date_str
                    
                    included_occupations_date0.append(current_occupation[job_titles])

                    if not G.has_node(node_current): # create the job-month node if it doesn't exist
                        G.add_node(node_current, date=date_str, occupation=current_occupation[job_titles], weight=1)
                    else: # otherwise just add 1 to its weight
                        G.nodes[node_current]['weight'] += 1
                    
                    if len(occupations_date1) > 0: # if the person has at least one job in the next month, we need to connect jobs at the current month to jobs at the next month
                        
                        included_occupations_date1 = []
                        for _, next_occupation in occupations_date1_df.iterrows(): # select 1 job occupation of that person in the next month
                            if next_occupation[job_titles] not in included_occupations_date1: # the person can have multiple jobs of the same occupation (in different companies) at the same time. This if makes it count only once
                                next_date_str = pd.to_datetime(str(date1)).strftime('%Y-%m-%d')
                                node_next = next_occupation[job_titles] + ' - ' + next_date_str

                                included_occupations_date1.append(next_occupation[job_titles])

                                # next_occupations = set(occupations_date1_df[job_titles])
                                intersec_occupations = occupations_date0 & occupations_date1 # will hold the jobs that the person had in common at month X and X+1
                                # all_occupations = occupations_date0 | occupations_date1 # will hold all jobs that the person had at month X and X+1
                                # different_occupations = all_occupations - intersec_occupations # will hold only the jobs that the person had at month X but not at X+1, and vice-versa
                                new_occupations = occupations_date1 - occupations_date0 # will hold the jobs that the person had at month X+1 but didn't at month X
                            
                                if not G.has_node(node_next): # if the job-month of month X+1 doesn't exist, create it
                                    G.add_node(node_next, date=next_date_str, occupation=next_occupation[job_titles], weight=0)   

                                # if the job is persistent, connect the persistent jobs
                                # connect job X in t0 to job X in t1
                                if current_occupation[job_titles] == next_occupation[job_titles]:
                                #and current_occupation[job_titles] in intersec_occupations \
                                #and next_occupation[job_titles] in intersec_occupations \
                                
                                    add_edge_update_label(G, node_current, node_next, person_id)

                                # if the job is new, connect all jobs at X to the new jobs at X+1
                                if next_occupation[job_titles] in new_occupations:
                                    add_edge_update_label(G, node_current, node_next, person_id)

                                # Now some really specific conditions so that no unnecessary edge is created
                                
                                # # Condition 1: person has 1 job in current month -> connect to all jobs in the next month
                                # if (len(occupations_date0) == 1):
                                #     add_edge_update_label(G, node_current, node_next, person_id)

                                # # Condition 2: person has >1 jobs in current month and only 1 job in the next month
                                # if (occupations_date0_df.shape[0] > 1) \
                                # and (occupations_date1_df.shape[0] == 1):
                                #     # Condition 2.1: one of the jobs in the current month is the same as the one in the next month -> connect only the identical jobs
                                #     if current_occupation[job_titles] in intersec_occupations \
                                #     and next_occupation[job_titles] in intersec_occupations:
                                #         add_edge_update_label(G, node_current, node_next, person_id)

                                #     # Condition 2.2: no job of the current month is the same as the one in the next month -> connect all current jobs to the next job
                                #     elif len(intersec_occupations) == 0:
                                #         add_edge_update_label(G, node_current, node_next, person_id)

                                # # Condition 3.1: person has >1 jobs in current month (X and Y) and >1 jobs in the next month (X and Z) AND at least one job is the same between the two months (X)
                                # if (occupations_date0_df.shape[0] > 1) \
                                # and (occupations_date1_df.shape[0] > 1) \
                                # and (len(intersec_occupations) >= 1):
                                    
                                #     # connect job X in t0 to job X in t1
                                #     if current_occupation[job_titles] in intersec_occupations \
                                #     and next_occupation[job_titles] in intersec_occupations \
                                #     and current_occupation[job_titles] == next_occupation[job_titles]:
                                #         add_edge_update_label(G, node_current, node_next, person_id)

                                #     # connect job Y in t0 to job Z in t1
                                #     elif current_occupation[job_titles] in different_occupations \
                                #     and next_occupation[job_titles] in different_occupations:
                                #         add_edge_update_label(G, node_current, node_next, person_id)

                                #     # connect job X in t0 to job Z in t1 only if 1) job Z didn't exist in t0, 2) job X doesn't exist in t4 (this accounts for the probation period, which is 3 months in Brazil - some people mantain two jobs for 3 months in order to make sure they are approved in the probation, so they can leave their old job. This condition makes sure that we catch this career trajectory "ahead of time".)
                                #     if current_occupation[job_titles] in intersec_occupations \
                                #     and next_occupation[job_titles] in different_occupations:# \
                                #     #and current_occupation[job_titles] not in occupations_date4:
                                #         add_edge_update_label(G, node_current, node_next, person_id)

                                # # Condition 3.2: person has >1 jobs in current month (X and Y) and >1 jobs in the next month (Z and K) AND no job is the same between the two months -> connect all
                                # if (occupations_date0_df.shape[0] > 1) \
                                # and (occupations_date1_df.shape[0] > 1)\
                                # and (len(intersec_occupations) == 0):
                                #     add_edge_update_label(G, node_current, node_next, person_id)

    return G


# edge_label = str(row['person_id'])
# G.add_node(node_next, date=next_date_str, occupation=next_occupation_row.iloc[0][job_titles], weight=0)
# G.add_edge(node_current, node_next, label=edge_label, weight=1, person_id=row['person_id'])

# df_in_test = single_experience_nodes_by_month_and_no_job_periods_df[single_experience_nodes_by_month_and_no_job_periods_df['person_id'].isin([109])] #[1, 3, 6, 20, 36, 42, 47, 75, 77, 85, 93, 109, 127])] # 137, 3 - 3, 5, 11, 13, 14, 137
gf_temporal_single_experience_nodes_traceable = create_directed_temporal_graph(single_experience_nodes_by_month_and_no_job_periods_df, 'model_classification', 'date')  # single_experience_nodes_by_month_and_no_job_periods_df

In [7]:
with open("gf_temporal_single_experience_nodes_traceable.pkl", "wb") as f:
    pickle.dump(gf_temporal_single_experience_nodes_traceable, f)

In [4]:
with open("gf_temporal_single_experience_nodes_traceable.pkl", "rb") as input_file:
    gf_temporal_single_experience_nodes_traceable = pickle.load(input_file)
gf_temporal_single_experience_nodes_traceable

In [5]:
print("Number of nodes:", gf_temporal_single_experience_nodes_traceable.number_of_nodes())
print("Number of edges:", gf_temporal_single_experience_nodes_traceable.number_of_edges())

# Number of nodes: 11574 -> 11574 -> 11574 -> 11574 -> 11873 -> 22854 -> 22569
# Number of edges: 20733 -> 20902 -> 20902 -> 20902 -> 21870 -> 38525 -> 38240

Number of nodes: 22569
Number of edges: 38240


### Now create a pyvis visualization for the original graph

In [8]:
pyvis_net = Network(height="1080px", width="100%", bgcolor='#FFFFFF', font_color='black', directed=True, notebook=True, select_menu=False, filter_menu=False)
pyvis_net.toggle_physics(False)\

# Set up positions
occupations = sorted(set(nx.get_node_attributes(gf_temporal_single_experience_nodes_traceable, 'occupation').values()))
dates = sorted(set(nx.get_node_attributes(gf_temporal_single_experience_nodes_traceable, 'date').values()))
occupation_positions = {occ: i * 400 for i, occ in enumerate(occupations)}
date_positions = {date: i * 400 for i, date in enumerate(dates)}

# Add nodes with fixed positions and scaled sizes
for node, node_attrs in gf_temporal_single_experience_nodes_traceable.nodes(data=True):
    x = date_positions[node_attrs['date']]
    y = occupation_positions[node_attrs['occupation']]
    weight = node_attrs.get('weight', 1)
    label = f"{node_attrs['occupation']}\n{node_attrs['date']}"
    title = label + f"\nNumber of people: {node_attrs.get('weight', 1)}"
    pyvis_net.add_node(node,  x=x, y=y, 
                       title=title,
                       color=f'rgba(0,0,0,1)',
                       size=np.log2(weight)*10,
                       occupation=node_attrs['occupation'], 
                       date=node_attrs['date'], 
                       number_of_people=weight)

# Add edges with scaled widths
for source, target, edge_attrs in gf_temporal_single_experience_nodes_traceable.edges(data=True):
    weight = edge_attrs.get('weight', 1)
    title = f"Number of people: {weight} - [{source.split('-')[0][:-1]}]->[{target.split('-')[0][:-1]}]"
    pyvis_net.add_edge(source, target,
                       title=title,
                       width=np.log10(weight)*5, 
                        arrowStrikethrough=False,
                       color=f'rgba(150,150,150,1)',
                       number_of_people=weight,
                       career_path_number=edge_attrs['label'],
                       person_id=edge_attrs['label'])

In [9]:
min_person_id = single_experience_nodes_by_month_and_no_job_periods_df['person_id'].min()
max_person_id = single_experience_nodes_by_month_and_no_job_periods_df['person_id'].max()


max_node_weight = max([node[1]['weight'] for node in gf_temporal_single_experience_nodes_traceable.nodes(data=True)])
max_edge_weight = max((edge_attrs.get('weight', 1) for _, _, edge_attrs in gf_temporal_single_experience_nodes_traceable.edges(data=True)), default=1)

def save_pyvis_html_with_filters(
    pyvis_net, html_filename, max_node_weight, max_edge_weight
):
    """
    Save the PyVis HTML graph with custom controls and filters.

    Args:
        pyvis_net: PyVis network instance
        html_filename: Filename to save the HTML graph
        node_data: List of node attributes
        edge_data: List of edge attributes
        min_person_id: Minimum person ID for dropdown
        max_person_id: Maximum person ID for dropdown
        max_node_weight: Maximum node weight
        max_edge_weight: Maximum edge weight
    """

    pyvis_net.set_edge_smooth("dynamic")
    pyvis_net.save_graph(html_filename)

    # Read the saved HTML file
    with open(html_filename, "r") as file:
        html_content = file.read()

    # Add filter controls
    filter_controls = create_filter_controls(
        max_node_weight, max_edge_weight
    )
    html_content = html_content.replace("</body>", filter_controls + "</body>")

    # Add custom CSS and JS
    custom_css = create_custom_css()
    custom_js = create_custom_js()
    handle_mover_js = create_handle_mover_js()
    year_marks_js = create_year_marks_js()
    occupation_marks_js = create_occupation_marks_js()
    html_content = html_content.replace("</head>", f"{custom_css}</head>")
    html_content = html_content.replace("</body>", f"{custom_js}</body>")
    html_content = html_content.replace("</body>", f"{handle_mover_js}</body>")
    html_content = html_content.replace("</body>", f"{year_marks_js}</body>")
    html_content = html_content.replace("</body>", f"{occupation_marks_js}</body>")
    

    # Save the updated HTML
    with open(html_filename, "w") as file:
        file.write(html_content)

def create_filter_controls(max_node_weight, max_edge_weight):
    """Generate HTML for filter controls."""
    return f"""
        <link href=\"https://cdnjs.cloudflare.com/ajax/libs/noUiSlider/14.6.3/nouislider.min.css\" rel=\"stylesheet\">
        <script src=\"https://cdnjs.cloudflare.com/ajax/libs/noUiSlider/14.6.3/nouislider.min.js\"></script>

        <div id="resizable-section">
        <!-- Resize Handle -->
        <div id="resize-handle"></div>

            <div style="display: flex; justify-content: space-between; height: calc(100% - 10px); padding: 10px;">
                <!-- Left Column -->
                <div id="left-column" style="flex: 1; margin-right: 20px;">
                    <!-- Edge Weight Slider -->
                    <label for="edge-weight-slider">Edge Weight Range:</label>
                    <div id="edge-weight-slider" class="slider" style="margin-top: 10px; margin-bottom: 20px;"></div>
                    <div>
                        <input type="number" id="edge-weight-slider-min-value" value="0" min="0" max="{max_edge_weight}" step="1" style="width: 80px;">
                        <span> - </span>
                        <input type="number" id="edge-weight-slider-max-value" value="{max_edge_weight}" min="0" max="{max_edge_weight}" step="1" style="width: 100px;">
                    </div>

                    <br>

                    <!-- Node Weight Slider -->
                    <label for="node-weight-slider">Node Weight Range:</label>
                    <div id="node-weight-slider" class="slider" style="margin-top: 10px; margin-bottom: 20px;"></div>
                    <div>
                        <input type="number" id="node-weight-slider-min-value" value="0" min="0" max="{max_node_weight}" step="1" style="width: 80px;">
                        <span> - </span>
                        <input type="number" id="node-weight-slider-max-value" value="{max_node_weight}" min="0" max="{max_node_weight}" step="1" style="width: 80px;">
                    </div>
                </div>

                <!-- Middle Column -->
                <div id="middle-column" style="flex: 1; margin-left: 20px; margin-right: 20px;">
                    <label>Edge Filters:</label>
                    <div style="margin-top: 10px; margin-bottom: 20px;">
                        <input type="checkbox" id="persistent-occupation" checked>
                        <label for="persistent-occupation">Persistent Occupation Edges</label>
                    </div>
                    <div style="margin-bottom: 20px;">
                        <input type="checkbox" id="transition-edges" checked>
                        <label for="transition-edges">Transition Edges</label>
                    </div>
                    <div>
                        <input type="checkbox" id="unregistered-job" checked>
                        <label for="unregistered-job">No job registered</label>
                    </div>
                </div>

                <!-- Right Column -->
                <div id="right-column" style="flex: 1; margin-left: 20px;">
                    <!-- Person ID Filter -->
                    <label for="person-id-dropdown">Person ID:</label>
                    <select id="person-id-dropdown" style="margin-top: 10px; margin-bottom: 20px; width: 100px;"></select>
                    <button id="add-button" style="margin-top: 10px;">Add</button>
                    <button id="clear-button" style="margin-top: 10px;">Clear</button>
                    <div id="selected-person-ids" style="margin-top: 10px;"></div>
                </div>
            </div>
        </div>

        """

def create_custom_css():
    """Generate custom CSS for the graph and filters."""
    return """
        <style>
            body {
                overflow-x: hidden; /* Disable horizontal scrolling */
            }

            #resizable-section {
                position: fixed;
                bottom: 0;
                left: 0;
                width: 100%;
                height: 20vh;
                background-color: #fff;
                z-index: 1000;
                box-shadow: 0 -4px 6px rgba(0, 0, 0, 0.1);
            }

            #resize-handle {
                position: absolute;
                top: -5px;
                left: 50%;
                transform: translateX(-50%);
                width: 100px;
                height: 10px;
                cursor: ns-resize;
                background-color: #ccc;
                border-radius: 5px;
            }

            #left-column, #right-column {
                overflow: hidden; /* Ensure no scrolling in individual columns */
            }

            .slider {
                width: 300px; /* Set consistent width for both sliders */
            }

            .year-label {
                position: absolute;
                color: black;
                transform: translateX(-50%);
                z-index: 10;
                top: 10px;
            }

            .occupation-label {
                position: absolute;
                color: black;
                transform: translateY(-50%);
                z-index: 10;
                right: 10px;
            }
        </style>
        """

def create_handle_mover_js():
    return f"""
        <script type="text/javascript">

            const resizableSection = document.getElementById('resizable-section');
            const resizeHandle = document.getElementById('resize-handle');

            let isDragging = false;

            resizeHandle.addEventListener('mousedown', (event) => {{
                isDragging = true;
                document.body.style.cursor = 'ns-resize';
            }});

            document.addEventListener('mousemove', (event) => {{
                if (!isDragging) return;

                const viewportHeight = window.innerHeight;
                const newHeight = viewportHeight - event.clientY;

                if (newHeight >= viewportHeight * 0.1 && newHeight <= viewportHeight * 0.5) {{ // Restrict height between 10% and 50% of viewport
                    resizableSection.style.height = `${{newHeight}}px`;
                }}
            }});

            document.addEventListener('mouseup', () => {{
                isDragging = false;
                document.body.style.cursor = 'default';
            }});
        </script>

    """

def create_year_marks_js():
    return """
        <script type="text/javascript">
            var yearMarkers = {};

            function parseDateUTC(dateString) {
                var parts = dateString.split('-');
                return new Date(Date.UTC(parts[0], parts[1] - 1, parts[2]));
            }

            function addYearLabels() {
                var container = document.getElementById('mynetwork');

                nodes.forEach(function(node) {
                    var date = parseDateUTC(node.date);
                    var year = date.getUTCFullYear();
                    var month = date.getUTCMonth();

                    if (!yearMarkers[year]) {
                        yearMarkers[year] = { first: null };
                    }
                    if (month === 0 && (!yearMarkers[year].first || date < parseDateUTC(yearMarkers[year].first.date))) {
                        yearMarkers[year].first = node; // Node with January date
                    }
                });

                Object.keys(yearMarkers).forEach(function(year) {
                    if (yearMarkers[year].first) {
                        var firstLabel = document.createElement('div');
                        firstLabel.className = 'year-label';
                        firstLabel.id = 'year-label-first-' + year;
                        firstLabel.innerHTML = year;
                        container.appendChild(firstLabel);
                    }
                });
            }

            function updateYearLabels() {
                var container = document.getElementById('mynetwork');
                var rect = container.getBoundingClientRect();

                Object.keys(yearMarkers).forEach(function(year) {
                    if (yearMarkers[year].first) {
                        var firstLabel = document.getElementById('year-label-first-' + year);
                        var firstNodePosition = network.getPositions([yearMarkers[year].first.id])[yearMarkers[year].first.id];
                        var firstCanvasPos = network.canvasToDOM({ x: firstNodePosition.x, y: 0 });

                        firstLabel.style.left = (firstCanvasPos.x + 15) + 'px';
                        firstLabel.style.top = '10px'; // Set the y position for the first label
                    }
                });
            }

            document.addEventListener("DOMContentLoaded", function() {
                addYearLabels();

                network.on('afterDrawing', function() {
                    updateYearLabels();
                });

                updateYearLabels(); // Initial positioning of the year labels
            });
        </script>
    """

def create_occupation_marks_js():
    return """
       <script type="text/javascript">
            var occupationMarkers = {}
            const visibleOccupations = new Set();

            function addOccupationLabels() {
                var container = document.getElementById('mynetwork');

                nodes.forEach(function(node) {
                    var occupation = node.occupation;

                    if (!occupationMarkers[occupation]) {
                        occupationMarkers[occupation] = { nodes: [] };
                    }
                    occupationMarkers[occupation].nodes.push(node);
                });

                Object.keys(occupationMarkers).forEach(function(occupation) {
                    if (occupationMarkers[occupation].nodes.length > 0) {
                        var label = document.createElement('div');
                        label.className = 'occupation-label';
                        label.id = 'occupation-label-' + occupation;
                        label.innerHTML = occupation;
                        container.appendChild(label);
                    }
                });
            }

            function updateOccupationLabels() {
                const container = document.getElementById('mynetwork').querySelector('canvas');
                const rect = container.getBoundingClientRect();

                Object.keys(occupationMarkers).forEach(function(occupation) {
                    const label = document.getElementById('occupation-label-' + occupation);
                    const nodes = occupationMarkers[occupation].nodes;

                    const positions = nodes.map(node => network.getPositions([node.id])[node.id]);
                    const avgY = positions.reduce((sum, pos) => sum + pos.y, 0) / positions.length;
                    const canvasPos = network.canvasToDOM({ x: 0, y: avgY });

                    const adjustedY = canvasPos.y + 15;

                    if (adjustedY >= label.offsetHeight && adjustedY <= rect.height) { // Check if the position is within bounds
                        label.style.display = 'block';
                        label.style.top = adjustedY + 'px';
                        label.style.right = '10px';
                    } else {
                        label.style.display = 'none';
                    }

                    if ((visibleOccupations.size === 0 || visibleOccupations.has(occupation)) && nodes.length > 0) { // If visibleOccupations is empty, show all labels
                        label.style.color = 'black';
                    } else {
                        label.style.color = '#DBDBDB';
                    }
                });
            }

            document.addEventListener("DOMContentLoaded", function() {

                addOccupationLabels();

                network.on('afterDrawing', function() {
                    updateOccupationLabels();
                });

                updateOccupationLabels(); // Initial positioning of the occupation labels
            });
        </script>
    """

def create_custom_js():
    """Generate custom JavaScript for interaction and filters."""
    return f"""
        <script>
                window.addEventListener('load', function() {{
                const minPersonId = {min_person_id};
                const maxPersonId = {max_person_id};
                const personIdDropdown = document.getElementById('person-id-dropdown');
                const addButton = document.getElementById('add-button');
                const clearButton = document.getElementById('clear-button');
                const selectedPersonIdsContainer = document.getElementById('selected-person-ids');
                const edgeWeightSlider = document.getElementById('edge-weight-slider');
                const nodeWeightSlider = document.getElementById('node-weight-slider');
                const persistentOccupationCheckbox = document.getElementById('persistent-occupation');
                const transitionEdgesCheckbox = document.getElementById('transition-edges');
                const unregisteredJobCheckbox = document.getElementById('unregistered-job');
                const colors = [
                    '#E69F00', '#56B4E9', '#009E73', '#0072B2', 
                    '#D55E00', '#CC79A7', '#999999', '#E69F00', '#56B4E9'
                ];


                let selectedPersonIds = [];
                let assignedColors = {{}};
                let nodeColors = {{}};
                let connectedNodes = new Set();
                let nodesWithingWeightRange = new Set();

                

                // ********************************************
                // Edge sliders number handling
                // ********************************************
                noUiSlider.create(edgeWeightSlider, {{
                    start: [0, parseInt({max_edge_weight})],
                    connect: true,
                    range: {{
                        'min': 0,
                        'max': parseInt({max_edge_weight})
                    }},
                    step: 1,
                    format: {{
                        to: function (value) {{
                            return Math.round(value); // Display as an integer
                        }},
                        from: function (value) {{
                            return Math.round(value); // Parse as an integer
                        }},
                    }},
                }});

                var edgeMinInput = document.getElementById('edge-weight-slider-min-value');
                var edgeMaxInput = document.getElementById('edge-weight-slider-max-value');

                edgeMinInput.addEventListener('change', function() {{
                    edgeWeightSlider.noUiSlider.set([this.value, null]);
                }});

                edgeMaxInput.addEventListener('change', function() {{
                    edgeWeightSlider.noUiSlider.set([null, this.value]);
                }});
                // ********************************************




                // ********************************************
                // Node sliders number handling
                // ********************************************
                noUiSlider.create(nodeWeightSlider, {{
                    start: [0, parseInt({max_node_weight})],
                    connect: true,
                    range: {{
                        'min': 0,
                        'max': parseInt({max_node_weight})
                    }},
                    step: 1,
                    format: {{
                        to: function (value) {{
                            return Math.round(value); // Display as an integer
                        }},
                        from: function (value) {{
                            return Math.round(value); // Parse as an integer
                        }},
                    }},
                }});

                var nodeMinInput = document.getElementById('node-weight-slider-min-value');
                var nodeMaxInput = document.getElementById('node-weight-slider-max-value');

                nodeMinInput.addEventListener('change', function() {{
                    nodeWeightSlider.noUiSlider.set([this.value, null]);
                }});

                nodeMaxInput.addEventListener('change', function() {{
                    nodeWeightSlider.noUiSlider.set([null, this.value]);
                }});
                // ********************************************

                


                // ********************************************
                // Edge checkbox handling
                // ********************************************
                persistentOccupationCheckbox.addEventListener('change', (event) => {{
                    persistentOccupationCheckbox.checked = event.target.checked;
                    updateEdgeVisibility();
                    updateNodeVisibility();
                }});

                transitionEdgesCheckbox.addEventListener('change', (event) => {{
                    transitionEdgesCheckbox.checked = event.target.checked;
                    updateEdgeVisibility();
                    updateNodeVisibility();
                }});

                unregisteredJobCheckbox.addEventListener('change', (event) => {{
                    unregisteredJobCheckbox.checked = event.target.checked;
                    updateEdgeVisibility();
                    updateNodeVisibility();
                }});
                // ********************************************




                // ***********************************************************************
                // Functions to generate a unique color for each person_id, up to 10 *****
                // ***********************************************************************
                function getColorForPersonId(personId) {{
                    if (!assignedColors[personId]) {{
                    const availableColors = colors.filter(color => !Object.values(assignedColors).includes(color));
                    if (availableColors.length > 0) {{
                        assignedColors[personId] = availableColors[0];
                    }} else {{
                        assignedColors[personId] = colors[Object.keys(assignedColors).length % colors.length];
                    }}
                    }}
                    return assignedColors[personId];
                }}

                // Function to generate a new color
                function generateNewColor() {{
                    const hue = Math.floor(Math.random() * 360);
                    const saturation = Math.floor(Math.random() * 50) + 50; // 50-100%
                    const lightness = Math.floor(Math.random() * 40) + 40; // 40-80%
                    return `hsl(${{hue}}, ${{saturation}}%, ${{lightness}}%)`;
                }}
                // *****************************************************************
                // *****************************************************************

                

                
                // *****************************************************************
                // Populate the person id dropdown *********************************
                // *****************************************************************
                for (let i = minPersonId; i <= maxPersonId; i++) {{
                    const option = document.createElement('option');
                    option.value = i;
                    option.text = i;
                    personIdDropdown.add(option);
                }}
                // *****************************************************************

                

                
                // *******************************************
                // Add new person to the Person ID filter ****
                // *******************************************
                addButton.addEventListener('click', function() {{
                    const selectedPersonId = parseFloat(personIdDropdown.value);
                    if (!selectedPersonIds.includes(selectedPersonId)) {{
                        selectedPersonIds.push(selectedPersonId);
                        updateSelectedPersonIds();
                        updateEdgeVisibility();
                        updateNodeVisibility();
                    }}
                }});
                // *******************************************

                // ****************************
                // Clear Person ID filter *****
                // ****************************
                clearButton.addEventListener('click', function() {{
                    selectedPersonIds = [];
                    updateSelectedPersonIds();
                    clearFilters();
                    visibleOccupations.clear();
                    updateEdgeVisibility();
                    updateNodeVisibility();
                }});
                // ****************************

                
                // ***********************************
                // Update selectedPersonIds **********
                // ***********************************
                function updateSelectedPersonIds() {{
                    selectedPersonIdsContainer.innerHTML = '';
                    selectedPersonIds.forEach(personId => {{
                        const personIdTag = document.createElement('div');
                        personIdTag.style.display = 'inline-block'; 
                        personIdTag.style.padding = '5px';
                        personIdTag.style.margin = '5px';
                        personIdTag.style.background = getColorForPersonId(personId);
                        personIdTag.style.color = 'white';
                        personIdTag.style.borderRadius = '3px';
                        personIdTag.innerHTML = personId + ' <span style="cursor: pointer;" onclick="removePersonId(' + personId + ')">&times;</span>';
                        selectedPersonIdsContainer.appendChild(personIdTag);
                    }});
                }}
                // ***********************************

                
                // ***********************************
                // Removal of a person Id ************
                // ***********************************
                window.removePersonId = removePersonId;

                function removePersonId(personId) {{
                    selectedPersonIds = selectedPersonIds.filter(id => id !== personId);
                    if (selectedPersonIds.length === 0) {{
                        nodeColors = {{}};
                    }}
                    updateSelectedPersonIds();
                    updateEdgeVisibility();
                    updateNodeVisibility();

                    
                }}
                // ***********************************

                
                // ***********************************
                // Remove PersonID filter ************
                // ***********************************
                function clearFilters() {{
                    // Reset selected person IDs
                    selectedPersonIds = [];

                    // Clear node colors and reset all nodes to black
                    nodeColors = {{}}; // Clear any previously assigned node colors

                    network.body.data.nodes.update(
                        network.body.data.nodes.get().map(function(node) {{
                            node.color = {{ background: 'black', border: 'black' }};
                            node.hidden = false; // Ensure all nodes are visible
                            return node;
                        }})
                    );

                    // Reset edge colors and visibility
                    network.body.data.edges.update(
                        network.body.data.edges.get().map(function(edge) {{
                            edge.color = {{ color: '#969696', highlight: '#969696', hover: '#969696' }};
                            edge.hidden = false; // Ensure all edges are visible
                            return edge;
                        }})
                    );

                    assignedColors = {{}}; // Clear assigned colors
                    updateEdgeVisibility();
                    updateNodeVisibility();
                }}
                // ********************************************

                

                

                // ***************************************************************
                // Event listener for Edge and Node sliders
                // ***************************************************************
                edgeWeightSlider.noUiSlider.on('update', function(values, handle) {{
                    edgeMinInput.value = values[0];
                    edgeMaxInput.value = values[1];
                    updateEdgeVisibility();
                    updateNodeVisibility();
                }});
                

                nodeWeightSlider.noUiSlider.on('update', function(values, handle) {{
                    nodeMinInput.value = values[0];
                    nodeMaxInput.value = values[1];
                    updateEdgeVisibility();
                    updateNodeVisibility();
                }});
                // ***************************************************************

                // ***************************************************************
                // Logic to update Edge Visibility
                // ***************************************************************
                function updateEdgeVisibility() {{
                    connectedNodes.clear()
                    network.body.data.edges.update(
                        network.body.data.edges.get().map(function (edge) {{
                            const careerPathNumbers = (edge.career_path_number || "")
                                .split(",")
                                .map((id) => parseFloat(id.trim()))
                                .filter((id) => !isNaN(id));

                            var edgeWeightSlider = document.getElementById('edge-weight-slider');
                            var minWeight = parseFloat(edgeWeightSlider.noUiSlider.get()[0]);
                            var maxWeight = parseFloat(edgeWeightSlider.noUiSlider.get()[1]);
                            var edgeWeight = edge.number_of_people || 0;
                            var origin_node_occupation = edge.from.split(' - ')[0];
                            var dest_node_occupation = edge.to.split(' - ')[0];

                            // Edge visibility conditions
                            const isCareerPathMatched =
                                selectedPersonIds.length === 0 || 
                                careerPathNumbers.some((id) => selectedPersonIds.includes(id));

                            const isWithinWeightRange = edgeWeight >= minWeight && edgeWeight <= maxWeight;

                            var isPersistentOccupation = origin_node_occupation===dest_node_occupation;
                            var isTransitioningOccupation = !isPersistentOccupation;
                            var isNoJobRegisteredEdge = 
                                (origin_node_occupation == 'no job registered') ||
                                (dest_node_occupation == 'no job registered');

                            // Set edge.hidden based on combined conditions
                            isEdgeMatched = isCareerPathMatched && isWithinWeightRange &&
                                (
                                    (
                                        (persistentOccupationCheckbox.checked && isPersistentOccupation) ||
                                        (transitionEdgesCheckbox.checked && isTransitioningOccupation)
                                    ) &&
                                  (unregisteredJobCheckbox.checked || !isNoJobRegisteredEdge)
                                );
                                

                            if (isEdgeMatched) {{
                                edge.hidden = false;
                                connectedNodes.add(edge.from);
                                connectedNodes.add(edge.to);

                                if (selectedPersonIds.length > 0) {{
                                    const matchedId = careerPathNumbers.find((id) =>
                                        selectedPersonIds.includes(id)
                                    );
                                    const edgeColor = getColorForPersonId(matchedId);
                                    edge.color = {{ color: edgeColor, highlight: edgeColor, hover: edgeColor }};
                                    nodeColors[edge.from] = edgeColor;
                                    nodeColors[edge.to] = edgeColor;
                                }}
                                else{{
                                    edge.color = {{ color: '#969696', highlight: '#969696', hover: '#969696' }};
                                }}
                            }}
                            else {{
                                edge.hidden = true;
                            }}

                            return edge;
                        }})
                    );
                }}
                // ***************************************************************




                // ***************************************************************
                // Logic to update Node Visibility
                // ***************************************************************
                function updateNodeVisibility() {{
                    nodesWithingWeightRange.clear()
                    visibleOccupations.clear()

                    var minWeight = parseFloat(nodeWeightSlider.noUiSlider.get()[0]);
                    var maxWeight = parseFloat(nodeWeightSlider.noUiSlider.get()[1]);

                    network.body.data.nodes.update(
                        network.body.data.nodes.get().map(function (node) {{
                            var nodeWeight = node.number_of_people || 0;
                            const isWithinWeightRange = nodeWeight >= minWeight && nodeWeight <= maxWeight;

                            if (isWithinWeightRange) {{
                                nodesWithingWeightRange.add(node.id)
                            }}


                            // Node visibility conditions
                            const isNodeMatched =
                                isWithinWeightRange && connectedNodes.has(node.id);

                            if (isNodeMatched) {{
                                node.hidden = false;
                                if (selectedPersonIds.length > 0) {{
                                    const nodeColor = nodeColors[node.id];
                                    node.color = {{ background: nodeColor, border: nodeColor }};
                                }}
                                else {{
                                    node.color = {{ background: 'black', border: 'black' }};
                                }}

                                visibleOccupations.add(node.occupation);
                            }} else {{
                                node.hidden = true;
                            }}
                            return node;
                        }})
                    );

                    // Update edges as well, since two nodes can disappear and it doesn't make sense to show their edges
                    network.body.data.edges.update(
                        network.body.data.edges.get().map(function (edge) {{
                            const isConnectedEdge = (connectedNodes.has(edge.from) && nodesWithingWeightRange.has(edge.from)) || 
                                                    (connectedNodes.has(edge.to) && (nodesWithingWeightRange.has(edge.to)));

                            if (!isConnectedEdge) {{
                                edge.hidden = true;
                            }}
                            
                            return edge;
                        }})

                        
                    );


                    updateOccupationLabels();
                }}
                // ***************************************************************


            }});
        </script>
        """

# Usage
save_pyvis_html_with_filters(
    pyvis_net, 
    "time_dimensional_single_experience_nodes_traceable.html",
    max_node_weight=max([node[1]["weight"] for node in gf_temporal_single_experience_nodes_traceable.nodes(data=True)]),
    max_edge_weight=max((edge_attrs.get("weight", 1) for _, _, edge_attrs in gf_temporal_single_experience_nodes_traceable.edges(data=True)), default=1)
)


### Small graph with interesting transitions

In [8]:
# Create a new graph which can be a simple Graph or DiGraph depending on what you are working with.
G = gf_temporal_single_experience_nodes_traceable

filtered_graph = type(G)()

# Add edges based on weight, differing occupations, and checking for 'no job registered'
for u, v, data in G.edges(data=True):
    if data.get('weight', 0) >= 5:
        if G.nodes[u].get('occupation') != G.nodes[v].get('occupation'):
            # Check if neither the source nor the destination node has 'no job registered' as their occupation
            if G.nodes[u].get('occupation') != 'no job registered' and G.nodes[v].get('occupation') != 'no job registered':
                filtered_graph.add_edge(u, v, **data)

# Copy node data
for node in filtered_graph.nodes():
    filtered_graph.nodes[node].update(G.nodes[node])

In [9]:
print("Number of nodes:", filtered_graph.number_of_nodes())
print("Number of edges:", filtered_graph.number_of_edges())

Number of nodes: 201
Number of edges: 176


In [10]:
with open("gf_FILTERED_single_experience_nodes_traceable.pkl", "wb") as f:
    pickle.dump(filtered_graph, f)

In [11]:
with open("gf_FILTERED_single_experience_nodes_traceable.pkl", "rb") as input_file:
    filtered_graph = pickle.load(input_file)
filtered_graph

### Create pyvis visualization for the filtered graph

In [12]:
pyvis_net_filtered = Network(height="1080px", width="100%", bgcolor='#FFFFFF', font_color='black', directed=True, notebook=True, select_menu=False, filter_menu=False)
pyvis_net_filtered.toggle_physics(False)\

# Set up positions
occupations = sorted(set(nx.get_node_attributes(filtered_graph, 'occupation').values()))
dates = sorted(set(nx.get_node_attributes(filtered_graph, 'date').values()))
occupation_positions = {occ: i * 400 for i, occ in enumerate(occupations)}
date_positions = {date: i * 400 for i, date in enumerate(dates)}

# Add nodes with fixed positions and scaled sizes
for node, node_attrs in filtered_graph.nodes(data=True):
    x = date_positions[node_attrs['date']]
    y = occupation_positions[node_attrs['occupation']]
    weight = node_attrs.get('weight', 1)
    label = f"{node_attrs['occupation']}\n{node_attrs['date']}"
    title = label + f"\nNumber of people: {node_attrs.get('weight', 1)}"
    pyvis_net_filtered.add_node(node,  x=x, y=y, 
                       label=label, title=title,
                       color=f'rgba(0,0,0,1)',
                       size=np.log2(weight)*10,
                       occupation=node_attrs['occupation'], 
                       date=node_attrs['date'], 
                       number_of_people=weight)

# Add edges with scaled widths
for source, target, edge_attrs in filtered_graph.edges(data=True):
    weight = edge_attrs.get('weight', 1)
    title = f"Number of people: {weight} - [{source.split('-')[0][:-1]}]->[{target.split('-')[0][:-1]}]"
    pyvis_net_filtered.add_edge(source, target,
                       title=title,
                       width=np.log10(weight)*20, 
                        arrowStrikethrough=False,
                       color=f'rgba(150,150,150,1)',
                       number_of_people=weight,
                       career_path_number=edge_attrs['label'],
                       person_id=edge_attrs['label'])

In [13]:
unique_person_ids = set()

for u, v, data in filtered_graph.edges(data=True):
    person_ids = data.get('person_id', '')
    if person_ids:
        ids = (id.strip() for id in person_ids.split(','))
        unique_person_ids.update(int(float(id)) for id in ids if id)

In [14]:
js_array = str(sorted(unique_person_ids)).replace(" ", "")


max_node_weight = max([node[1]['weight'] for node in gf_temporal_single_experience_nodes_traceable.nodes(data=True)])
max_edge_weight = max((edge_attrs.get('weight', 1) for _, _, edge_attrs in gf_temporal_single_experience_nodes_traceable.edges(data=True)), default=1)

def save_pyvis_html_with_filters(
    pyvis_net, html_filename, max_node_weight, max_edge_weight
):
    """
    Save the PyVis HTML graph with custom controls and filters.

    Args:
        pyvis_net: PyVis network instance
        html_filename: Filename to save the HTML graph
        node_data: List of node attributes
        edge_data: List of edge attributes
        min_person_id: Minimum person ID for dropdown
        max_person_id: Maximum person ID for dropdown
        max_node_weight: Maximum node weight
        max_edge_weight: Maximum edge weight
    """

    pyvis_net.set_edge_smooth("dynamic")
    pyvis_net.save_graph(html_filename)

    # Read the saved HTML file
    with open(html_filename, "r") as file:
        html_content = file.read()

    # Add filter controls
    filter_controls = create_filter_controls(
        max_node_weight, max_edge_weight
    )
    html_content = html_content.replace("</body>", filter_controls + "</body>")

    # Add custom CSS and JS
    custom_css = create_custom_css()
    custom_js = create_custom_js()
    handle_mover_js = create_handle_mover_js()
    year_marks_js = create_year_marks_js()
    occupation_marks_js = create_occupation_marks_js()
    html_content = html_content.replace("</head>", f"{custom_css}</head>")
    html_content = html_content.replace("</body>", f"{custom_js}</body>")
    html_content = html_content.replace("</body>", f"{handle_mover_js}</body>")
    html_content = html_content.replace("</body>", f"{year_marks_js}</body>")
    html_content = html_content.replace("</body>", f"{occupation_marks_js}</body>")
    

    # Save the updated HTML
    with open(html_filename, "w") as file:
        file.write(html_content)

def create_filter_controls(max_node_weight, max_edge_weight):
    """Generate HTML for filter controls."""
    return f"""
        <link href=\"https://cdnjs.cloudflare.com/ajax/libs/noUiSlider/14.6.3/nouislider.min.css\" rel=\"stylesheet\">
        <script src=\"https://cdnjs.cloudflare.com/ajax/libs/noUiSlider/14.6.3/nouislider.min.js\"></script>

        <div id="resizable-section">
        <!-- Resize Handle -->
        <div id="resize-handle"></div>

            <div style="display: flex; justify-content: space-between; height: calc(100% - 10px); padding: 10px;">
                <!-- Left Column -->
                <div id="left-column" style="flex: 1; margin-right: 20px;">
                    <!-- Edge Weight Slider -->
                    <label for="edge-weight-slider">Edge Weight Range:</label>
                    <div id="edge-weight-slider" class="slider" style="margin-top: 10px; margin-bottom: 20px;"></div>
                    <div>
                        <input type="number" id="edge-weight-slider-min-value" value="0" min="0" max="{max_edge_weight}" step="1" style="width: 80px;">
                        <span> - </span>
                        <input type="number" id="edge-weight-slider-max-value" value="{max_edge_weight}" min="0" max="{max_edge_weight}" step="1" style="width: 100px;">
                    </div>

                    <br>

                    <!-- Node Weight Slider -->
                    <label for="node-weight-slider">Node Weight Range:</label>
                    <div id="node-weight-slider" class="slider" style="margin-top: 10px; margin-bottom: 20px;"></div>
                    <div>
                        <input type="number" id="node-weight-slider-min-value" value="0" min="0" max="{max_node_weight}" step="1" style="width: 80px;">
                        <span> - </span>
                        <input type="number" id="node-weight-slider-max-value" value="{max_node_weight}" min="0" max="{max_node_weight}" step="1" style="width: 80px;">
                    </div>
                </div>

                <!-- Middle Column -->
                <div id="middle-column" style="flex: 1; margin-left: 20px; margin-right: 20px;">
                    <label>Edge Filters:</label>
                    <div style="margin-top: 10px; margin-bottom: 20px;">
                        <input type="checkbox" id="persistent-occupation" checked>
                        <label for="persistent-occupation">Persistent Occupation Edges</label>
                    </div>
                    <div style="margin-bottom: 20px;">
                        <input type="checkbox" id="transition-edges" checked>
                        <label for="transition-edges">Transition Edges</label>
                    </div>
                    <div>
                        <input type="checkbox" id="unregistered-job" checked>
                        <label for="unregistered-job">No job registered</label>
                    </div>
                </div>

                <!-- Right Column -->
                <div id="right-column" style="flex: 1; margin-left: 20px;">
                    <!-- Person ID Filter -->
                    <label for="person-id-dropdown">Person ID:</label>
                    <select id="person-id-dropdown" style="margin-top: 10px; margin-bottom: 20px; width: 100px;"></select>
                    <button id="add-button" style="margin-top: 10px;">Add</button>
                    <button id="clear-button" style="margin-top: 10px;">Clear</button>
                    <div id="selected-person-ids" style="margin-top: 10px;"></div>
                </div>
            </div>
        </div>

        """

def create_custom_css():
    """Generate custom CSS for the graph and filters."""
    return """
        <style>
            body {
                overflow-x: hidden; /* Disable horizontal scrolling */
            }

            #resizable-section {
                position: fixed;
                bottom: 0;
                left: 0;
                width: 100%;
                height: 20vh;
                background-color: #fff;
                z-index: 1000;
                box-shadow: 0 -4px 6px rgba(0, 0, 0, 0.1);
            }

            #resize-handle {
                position: absolute;
                top: -5px;
                left: 50%;
                transform: translateX(-50%);
                width: 100px;
                height: 10px;
                cursor: ns-resize;
                background-color: #ccc;
                border-radius: 5px;
            }

            #left-column, #right-column {
                overflow: hidden; /* Ensure no scrolling in individual columns */
            }

            .slider {
                width: 300px; /* Set consistent width for both sliders */
            }

            .year-label {
                position: absolute;
                color: black;
                transform: translateX(-50%);
                z-index: 10;
                top: 10px;
            }

            .occupation-label {
                position: absolute;
                color: black;
                transform: translateY(-50%);
                z-index: 10;
                right: 10px;
            }
        </style>
        """

def create_handle_mover_js():
    return f"""
        <script type="text/javascript">

            const resizableSection = document.getElementById('resizable-section');
            const resizeHandle = document.getElementById('resize-handle');

            let isDragging = false;

            resizeHandle.addEventListener('mousedown', (event) => {{
                isDragging = true;
                document.body.style.cursor = 'ns-resize';
            }});

            document.addEventListener('mousemove', (event) => {{
                if (!isDragging) return;

                const viewportHeight = window.innerHeight;
                const newHeight = viewportHeight - event.clientY;

                if (newHeight >= viewportHeight * 0.1 && newHeight <= viewportHeight * 0.5) {{ // Restrict height between 10% and 50% of viewport
                    resizableSection.style.height = `${{newHeight}}px`;
                }}
            }});

            document.addEventListener('mouseup', () => {{
                isDragging = false;
                document.body.style.cursor = 'default';
            }});
        </script>

    """

def create_year_marks_js():
    return """
        <script type="text/javascript">
            var yearMarkers = {};

            function parseDateUTC(dateString) {
                var parts = dateString.split('-');
                return new Date(Date.UTC(parts[0], parts[1] - 1, parts[2]));
            }

            function addYearLabels() {
                var container = document.getElementById('mynetwork');

                nodes.forEach(function(node) {
                    var date = parseDateUTC(node.date);
                    var year = date.getUTCFullYear();
                    var month = date.getUTCMonth();

                    if (!yearMarkers[year]) {
                        yearMarkers[year] = { first: null };
                    }
                    if (month === 0 && (!yearMarkers[year].first || date < parseDateUTC(yearMarkers[year].first.date))) {
                        yearMarkers[year].first = node; // Node with January date
                    }
                });

                Object.keys(yearMarkers).forEach(function(year) {
                    if (yearMarkers[year].first) {
                        var firstLabel = document.createElement('div');
                        firstLabel.className = 'year-label';
                        firstLabel.id = 'year-label-first-' + year;
                        firstLabel.innerHTML = year;
                        container.appendChild(firstLabel);
                    }
                });
            }

            function updateYearLabels() {
                var container = document.getElementById('mynetwork');
                var rect = container.getBoundingClientRect();

                Object.keys(yearMarkers).forEach(function(year) {
                    if (yearMarkers[year].first) {
                        var firstLabel = document.getElementById('year-label-first-' + year);
                        var firstNodePosition = network.getPositions([yearMarkers[year].first.id])[yearMarkers[year].first.id];
                        var firstCanvasPos = network.canvasToDOM({ x: firstNodePosition.x, y: 0 });

                        firstLabel.style.left = (firstCanvasPos.x + 15) + 'px';
                        firstLabel.style.top = '10px'; // Set the y position for the first label
                    }
                });
            }

            document.addEventListener("DOMContentLoaded", function() {
                addYearLabels();

                network.on('afterDrawing', function() {
                    updateYearLabels();
                });

                updateYearLabels(); // Initial positioning of the year labels
            });
        </script>
    """

def create_occupation_marks_js():
    return """
       <script type="text/javascript">
            var occupationMarkers = {}
            const visibleOccupations = new Set();

            function addOccupationLabels() {
                var container = document.getElementById('mynetwork');

                nodes.forEach(function(node) {
                    var occupation = node.occupation;

                    if (!occupationMarkers[occupation]) {
                        occupationMarkers[occupation] = { nodes: [] };
                    }
                    occupationMarkers[occupation].nodes.push(node);
                });

                Object.keys(occupationMarkers).forEach(function(occupation) {
                    if (occupationMarkers[occupation].nodes.length > 0) {
                        var label = document.createElement('div');
                        label.className = 'occupation-label';
                        label.id = 'occupation-label-' + occupation;
                        label.innerHTML = occupation;
                        container.appendChild(label);
                    }
                });
            }

            function updateOccupationLabels() {
                const container = document.getElementById('mynetwork').querySelector('canvas');
                const rect = container.getBoundingClientRect();

                Object.keys(occupationMarkers).forEach(function(occupation) {
                    const label = document.getElementById('occupation-label-' + occupation);
                    const nodes = occupationMarkers[occupation].nodes;

                    const positions = nodes.map(node => network.getPositions([node.id])[node.id]);
                    const avgY = positions.reduce((sum, pos) => sum + pos.y, 0) / positions.length;
                    const canvasPos = network.canvasToDOM({ x: 0, y: avgY });

                    const adjustedY = canvasPos.y + 15;

                    if (adjustedY >= label.offsetHeight && adjustedY <= rect.height) { // Check if the position is within bounds
                        label.style.display = 'block';
                        label.style.top = adjustedY + 'px';
                        label.style.right = '10px';
                    } else {
                        label.style.display = 'none';
                    }

                    if ((visibleOccupations.size === 0 || visibleOccupations.has(occupation)) && nodes.length > 0) { // If visibleOccupations is empty, show all labels
                        label.style.color = 'black';
                    } else {
                        label.style.color = '#DBDBDB';
                    }
                });
            }

            document.addEventListener("DOMContentLoaded", function() {

                addOccupationLabels();

                network.on('afterDrawing', function() {
                    updateOccupationLabels();
                });

                updateOccupationLabels(); // Initial positioning of the occupation labels
            });
        </script>
    """

def create_custom_js():
    """Generate custom JavaScript for interaction and filters."""
    return f"""
        <script>
                window.addEventListener('load', function() {{
                const minPersonId = {min_person_id};
                const maxPersonId = {max_person_id};
                const personIdDropdown = document.getElementById('person-id-dropdown');
                const addButton = document.getElementById('add-button');
                const clearButton = document.getElementById('clear-button');
                const selectedPersonIdsContainer = document.getElementById('selected-person-ids');
                const edgeWeightSlider = document.getElementById('edge-weight-slider');
                const nodeWeightSlider = document.getElementById('node-weight-slider');
                const persistentOccupationCheckbox = document.getElementById('persistent-occupation');
                const transitionEdgesCheckbox = document.getElementById('transition-edges');
                const unregisteredJobCheckbox = document.getElementById('unregistered-job');
                const colors = [
                    '#E69F00', '#56B4E9', '#009E73', '#0072B2', 
                    '#D55E00', '#CC79A7', '#999999', '#E69F00', '#56B4E9'
                ];


                let selectedPersonIds = [];
                let assignedColors = {{}};
                let nodeColors = {{}};
                let connectedNodes = new Set();
                let nodesWithingWeightRange = new Set();

                

                // ********************************************
                // Edge sliders number handling
                // ********************************************
                noUiSlider.create(edgeWeightSlider, {{
                    start: [0, parseInt({max_edge_weight})],
                    connect: true,
                    range: {{
                        'min': 0,
                        'max': parseInt({max_edge_weight})
                    }},
                    step: 1,
                    format: {{
                        to: function (value) {{
                            return Math.round(value); // Display as an integer
                        }},
                        from: function (value) {{
                            return Math.round(value); // Parse as an integer
                        }},
                    }},
                }});

                var edgeMinInput = document.getElementById('edge-weight-slider-min-value');
                var edgeMaxInput = document.getElementById('edge-weight-slider-max-value');

                edgeMinInput.addEventListener('change', function() {{
                    edgeWeightSlider.noUiSlider.set([this.value, null]);
                }});

                edgeMaxInput.addEventListener('change', function() {{
                    edgeWeightSlider.noUiSlider.set([null, this.value]);
                }});
                // ********************************************




                // ********************************************
                // Node sliders number handling
                // ********************************************
                noUiSlider.create(nodeWeightSlider, {{
                    start: [0, parseInt({max_node_weight})],
                    connect: true,
                    range: {{
                        'min': 0,
                        'max': parseInt({max_node_weight})
                    }},
                    step: 1,
                    format: {{
                        to: function (value) {{
                            return Math.round(value); // Display as an integer
                        }},
                        from: function (value) {{
                            return Math.round(value); // Parse as an integer
                        }},
                    }},
                }});

                var nodeMinInput = document.getElementById('node-weight-slider-min-value');
                var nodeMaxInput = document.getElementById('node-weight-slider-max-value');

                nodeMinInput.addEventListener('change', function() {{
                    nodeWeightSlider.noUiSlider.set([this.value, null]);
                }});

                nodeMaxInput.addEventListener('change', function() {{
                    nodeWeightSlider.noUiSlider.set([null, this.value]);
                }});
                // ********************************************

                


                // ********************************************
                // Edge checkbox handling
                // ********************************************
                persistentOccupationCheckbox.addEventListener('change', (event) => {{
                    persistentOccupationCheckbox.checked = event.target.checked;
                    updateEdgeVisibility();
                    updateNodeVisibility();
                }});

                transitionEdgesCheckbox.addEventListener('change', (event) => {{
                    transitionEdgesCheckbox.checked = event.target.checked;
                    updateEdgeVisibility();
                    updateNodeVisibility();
                }});

                unregisteredJobCheckbox.addEventListener('change', (event) => {{
                    unregisteredJobCheckbox.checked = event.target.checked;
                    updateEdgeVisibility();
                    updateNodeVisibility();
                }});
                // ********************************************




                // ***********************************************************************
                // Functions to generate a unique color for each person_id, up to 10 *****
                // ***********************************************************************
                function getColorForPersonId(personId) {{
                    if (!assignedColors[personId]) {{
                    const availableColors = colors.filter(color => !Object.values(assignedColors).includes(color));
                    if (availableColors.length > 0) {{
                        assignedColors[personId] = availableColors[0];
                    }} else {{
                        assignedColors[personId] = colors[Object.keys(assignedColors).length % colors.length];
                    }}
                    }}
                    return assignedColors[personId];
                }}

                // Function to generate a new color
                function generateNewColor() {{
                    const hue = Math.floor(Math.random() * 360);
                    const saturation = Math.floor(Math.random() * 50) + 50; // 50-100%
                    const lightness = Math.floor(Math.random() * 40) + 40; // 40-80%
                    return `hsl(${{hue}}, ${{saturation}}%, ${{lightness}}%)`;
                }}
                // *****************************************************************
                // *****************************************************************

                

                
                // *****************************************************************
                // Populate the person id dropdown *********************************
                // *****************************************************************
                const uniquePersonIds = {js_array}
                uniquePersonIds.forEach(id => {{
                    const option = document.createElement('option');
                    option.value = id;
                    option.text = id;
                    personIdDropdown.add(option);
                }});
                // *****************************************************************

                

                
                // *******************************************
                // Add new person to the Person ID filter ****
                // *******************************************
                addButton.addEventListener('click', function() {{
                    const selectedPersonId = parseFloat(personIdDropdown.value);
                    if (!selectedPersonIds.includes(selectedPersonId)) {{
                        selectedPersonIds.push(selectedPersonId);
                        updateSelectedPersonIds();
                        updateEdgeVisibility();
                        updateNodeVisibility();
                    }}
                }});
                // *******************************************

                // ****************************
                // Clear Person ID filter *****
                // ****************************
                clearButton.addEventListener('click', function() {{
                    selectedPersonIds = [];
                    updateSelectedPersonIds();
                    clearFilters();
                    visibleOccupations.clear();
                    updateEdgeVisibility();
                    updateNodeVisibility();
                }});
                // ****************************

                
                // ***********************************
                // Update selectedPersonIds **********
                // ***********************************
                function updateSelectedPersonIds() {{
                    selectedPersonIdsContainer.innerHTML = '';
                    selectedPersonIds.forEach(personId => {{
                        const personIdTag = document.createElement('div');
                        personIdTag.style.display = 'inline-block'; 
                        personIdTag.style.padding = '5px';
                        personIdTag.style.margin = '5px';
                        personIdTag.style.background = getColorForPersonId(personId);
                        personIdTag.style.color = 'white';
                        personIdTag.style.borderRadius = '3px';
                        personIdTag.innerHTML = personId + ' <span style="cursor: pointer;" onclick="removePersonId(' + personId + ')">&times;</span>';
                        selectedPersonIdsContainer.appendChild(personIdTag);
                    }});
                }}
                // ***********************************

                
                // ***********************************
                // Removal of a person Id ************
                // ***********************************
                window.removePersonId = removePersonId;

                function removePersonId(personId) {{
                    selectedPersonIds = selectedPersonIds.filter(id => id !== personId);
                    if (selectedPersonIds.length === 0) {{
                        nodeColors = {{}};
                    }}
                    updateSelectedPersonIds();
                    updateEdgeVisibility();
                    updateNodeVisibility();

                    
                }}
                // ***********************************

                
                // ***********************************
                // Remove PersonID filter ************
                // ***********************************
                function clearFilters() {{
                    // Reset selected person IDs
                    selectedPersonIds = [];

                    // Clear node colors and reset all nodes to black
                    nodeColors = {{}}; // Clear any previously assigned node colors

                    network.body.data.nodes.update(
                        network.body.data.nodes.get().map(function(node) {{
                            node.color = {{ background: 'black', border: 'black' }};
                            node.hidden = false; // Ensure all nodes are visible
                            return node;
                        }})
                    );

                    // Reset edge colors and visibility
                    network.body.data.edges.update(
                        network.body.data.edges.get().map(function(edge) {{
                            edge.color = {{ color: '#969696', highlight: '#969696', hover: '#969696' }};
                            edge.hidden = false; // Ensure all edges are visible
                            return edge;
                        }})
                    );

                    assignedColors = {{}}; // Clear assigned colors
                    updateEdgeVisibility();
                    updateNodeVisibility();
                }}
                // ********************************************

                

                

                // ***************************************************************
                // Event listener for Edge and Node sliders
                // ***************************************************************
                edgeWeightSlider.noUiSlider.on('update', function(values, handle) {{
                    edgeMinInput.value = values[0];
                    edgeMaxInput.value = values[1];
                    updateEdgeVisibility();
                    updateNodeVisibility();
                }});
                

                nodeWeightSlider.noUiSlider.on('update', function(values, handle) {{
                    nodeMinInput.value = values[0];
                    nodeMaxInput.value = values[1];
                    updateEdgeVisibility();
                    updateNodeVisibility();
                }});
                // ***************************************************************

                // ***************************************************************
                // Logic to update Edge Visibility
                // ***************************************************************
                function updateEdgeVisibility() {{
                    connectedNodes.clear()
                    network.body.data.edges.update(
                        network.body.data.edges.get().map(function (edge) {{
                            const careerPathNumbers = (edge.career_path_number || "")
                                .split(",")
                                .map((id) => parseFloat(id.trim()))
                                .filter((id) => !isNaN(id));

                            var edgeWeightSlider = document.getElementById('edge-weight-slider');
                            var minWeight = parseFloat(edgeWeightSlider.noUiSlider.get()[0]);
                            var maxWeight = parseFloat(edgeWeightSlider.noUiSlider.get()[1]);
                            var edgeWeight = edge.number_of_people || 0;
                            var origin_node_occupation = edge.from.split(' - ')[0];
                            var dest_node_occupation = edge.to.split(' - ')[0];

                            // Edge visibility conditions
                            const isCareerPathMatched =
                                selectedPersonIds.length === 0 || 
                                careerPathNumbers.some((id) => selectedPersonIds.includes(id));

                            const isWithinWeightRange = edgeWeight >= minWeight && edgeWeight <= maxWeight;

                            var isPersistentOccupation = origin_node_occupation===dest_node_occupation;
                            var isTransitioningOccupation = !isPersistentOccupation;
                            var isNoJobRegisteredEdge = 
                                (origin_node_occupation == 'no job registered') ||
                                (dest_node_occupation == 'no job registered');

                            // Set edge.hidden based on combined conditions
                            isEdgeMatched = isCareerPathMatched && isWithinWeightRange &&
                                (
                                    (
                                        (persistentOccupationCheckbox.checked && isPersistentOccupation) ||
                                        (transitionEdgesCheckbox.checked && isTransitioningOccupation)
                                    ) &&
                                  (unregisteredJobCheckbox.checked || !isNoJobRegisteredEdge)
                                );
                                

                            if (isEdgeMatched) {{
                                edge.hidden = false;
                                connectedNodes.add(edge.from);
                                connectedNodes.add(edge.to);

                                if (selectedPersonIds.length > 0) {{
                                    const matchedId = careerPathNumbers.find((id) =>
                                        selectedPersonIds.includes(id)
                                    );
                                    const edgeColor = getColorForPersonId(matchedId);
                                    edge.color = {{ color: edgeColor, highlight: edgeColor, hover: edgeColor }};
                                    nodeColors[edge.from] = edgeColor;
                                    nodeColors[edge.to] = edgeColor;
                                }}
                                else{{
                                    edge.color = {{ color: '#969696', highlight: '#969696', hover: '#969696' }};
                                }}
                            }}
                            else {{
                                edge.hidden = true;
                            }}

                            return edge;
                        }})
                    );
                }}
                // ***************************************************************




                // ***************************************************************
                // Logic to update Node Visibility
                // ***************************************************************
                function updateNodeVisibility() {{
                    nodesWithingWeightRange.clear()
                    visibleOccupations.clear()

                    var minWeight = parseFloat(nodeWeightSlider.noUiSlider.get()[0]);
                    var maxWeight = parseFloat(nodeWeightSlider.noUiSlider.get()[1]);

                    network.body.data.nodes.update(
                        network.body.data.nodes.get().map(function (node) {{
                            var nodeWeight = node.number_of_people || 0;
                            const isWithinWeightRange = nodeWeight >= minWeight && nodeWeight <= maxWeight;

                            if (isWithinWeightRange) {{
                                nodesWithingWeightRange.add(node.id)
                            }}


                            // Node visibility conditions
                            const isNodeMatched =
                                isWithinWeightRange && connectedNodes.has(node.id);

                            if (isNodeMatched) {{
                                node.hidden = false;
                                if (selectedPersonIds.length > 0) {{
                                    const nodeColor = nodeColors[node.id];
                                    node.color = {{ background: nodeColor, border: nodeColor }};
                                }}
                                else {{
                                    node.color = {{ background: 'black', border: 'black' }};
                                }}

                                visibleOccupations.add(node.occupation);
                            }} else {{
                                node.hidden = true;
                            }}
                            return node;
                        }})
                    );

                    // Update edges as well, since two nodes can disappear and it doesn't make sense to show their edges
                    network.body.data.edges.update(
                        network.body.data.edges.get().map(function (edge) {{
                            const isConnectedEdge = (connectedNodes.has(edge.from) && nodesWithingWeightRange.has(edge.from)) || 
                                                    (connectedNodes.has(edge.to) && (nodesWithingWeightRange.has(edge.to)));

                            if (!isConnectedEdge) {{
                                edge.hidden = true;
                            }}
                            
                            return edge;
                        }})

                        
                    );


                    updateOccupationLabels();
                }}
                // ***************************************************************


            }});
        </script>
        """

# Usage
save_pyvis_html_with_filters(
    pyvis_net_filtered, 
    "time_dimensional_single_experience_nodes_traceable_FILTERED.html",
    max_node_weight=max([node[1]["weight"] for node in gf_temporal_single_experience_nodes_traceable.nodes(data=True)]),
    max_edge_weight=max((edge_attrs.get("weight", 1) for _, _, edge_attrs in gf_temporal_single_experience_nodes_traceable.edges(data=True)), default=1)
)
